### Remove Broken Images

In [ ]:
from glob import glob
from tqdm import tqdm
import cv2
import os

img_files = glob('autodl-tmp/ACCV_384/*/*')
for file in tqdm(img_files):
    img = cv2.imread(file)
    if img is None:
        print('None:', file)
    elif img.sum()<100:
        print('Remove:', file)
        os.remove(file)
    elif img.shape[2]!=3:
        print('Png:', file)

### Make csv file

In [ ]:
from glob import glob
from tqdm import tqdm
import os

all_ids = []
all_labels = []
all_cls = glob('autodl-tmp/ACCV_384/*')
for c in all_cls:
    files = glob(os.path.join(c, '*'))
    all_ids.extend(files)
    all_lables.extend([int(cl.split('/')[-2]) for cl in files])
    
df_dict = {'image_files': all_ids,
           'lables': all_labels,}
df = pd.DataFrame(df_dict)
df.to_csv('ACCV_384.csv', index=False)
df.head()

### Statistic

In [ ]:
import pandas as pd
df = pd.read_csv('autodl-tmp/ACCV_384.csv')
counts = df['labels'].value_counts().to_list()
print('max_count:', max(counts))
print('min_count:', min(counts))

### Resample

In [ ]:
import pandas as pd
from tqdm import tqdm
import math
min_ = 211
df = pd.read_csv('autodl-tmp/ACCV_384_fold.csv')
new_groups = []
for label, group in tqdm(df.groupby('labels')):
    group = group.reset_index(drop=True)
    if len(group)<min_:
        n = min_//len(group)
        for i in range(n):
            new_groups.append(group)
        if min_%len(group)!=0:
            new_groups.append(group.sample(min_%len(group)))
    else:
        new_groups.append(group)
new_df = pd.concat(new_groups, axis=0).reset_index(drop=True)
print('num:', df.shape[0], 'classes:', df['labels'].nunique())
print('----->>>')
print('num:', new_df.shape[0], 'classes:', new_df['labels'].nunique())
new_df.to_csv('autodl-tmp/ACCV_384_balance_fold.csv', index=False)
new_df.head(5)

### StratifiedKfold

In [ ]:
from glob import glob
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
df = pd.read_csv('autodl-tmp/ACCV_384_balance_fold.csv')
df['fold'] = -1
split = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=999).split(df, df['labels']))
for fold, (train_idx, valid_idx) in enumerate(split):
    df.loc[valid_idx, 'fold'] = fold
df.to_csv('autodl-tmp/ACCV_384_balance_fold.csv', index=False)
print(df.shape[0], df.nunique())
df.head(10)